Class to accumulate whois records and risk factors.
They will memoized to reduce requested services for more data.

1. Use a database to store the results for use in various programs.
2. Use an ipnetwork as key
3. Store company name with key
4. When a class instance is created, it will load db into dictionary.
5. The instance is normally readonly, but can be writeable. That means new addresses
will be added if they are not in the database and become permanent parts.



In [1]:
import sys
import pandas as pd
import numpy as np
import ipaddress
import dbm
import pickle
from sortedcontainers import SortedDict
import requests
from bs4 import BeautifulSoup
import json
import os

import pprint
pp = pprint.PrettyPrinter()

class Debug():
    """ Print debug messages if active """
    def __init__(self, set=1):
        self._set = set
    def prt(self, str):
        if self._set:
            sys.stderr.write(str)
    def set(self):
        self._set = 1
    def unset(self):
        self._set = 0
debug = Debug()


def get_risk(ip_string):
    # Return risk factors from scamalytics into a dict
    #     {"ip": ? , 
    #      "score": ?, 
    #      "risk": ?, 
    #      "risk_comment: ?"}

    # Fetch the complete record from scamalytics restful api
    # ip_string ... make request by ip address as a string

    html_text = ""
    url = "https://scamalytics.com/ip/" + ip_string
    html_text = requests.get(url).text

#     soup = BeautifulSoup(html_text, 'lxml')
    soup = BeautifulSoup(html_text, 'xml')
    
    # Tag=pre
    result = json.loads(soup.pre.string)
    
    # The comment is in the body of an unlabelled div. Used the css class to find.
    # Remove special UTF-8 character \U200b, a zero width space.
#     result["risk_comment"] = soup.find_all("div", class_="panel_body")[0].get_text().replace("\u200b","")
    result["risk_comment"] = ""
    
    return result

def get_arin(ip_string):
    '''Return dict for the net_address that contains this ip_string
        {"cidr": ?,
         {"organization": ? ,
          "handle": ? ,
          "asn": ?,
          "city": ? ,
          "address" : ? ,
          "postalcode": ? ,
          "countrycode": ? ,
          "state": ? ,
          "country": ? ,
         }
    '''
    
    fillna = lambda x: "" if not x else x.string if not isinstance(x, str) else x
    
    # Fetch the complete record from arin restful api
    # Ref: https://www.arin.net/resources/registry/whois/rws/api/#networks-and-asns
    # ip ... make request by ip address as a string
    # pft .. get full record
    
    url       = "http://whois.arin.net/rest/ip/" + ip_string + "/pft"
    html_text = ""
    try:
        html_text = requests.get(url).text
    except:
        return None

    # Parse html into a hierarchy using BeautifulSoup 
#     soup = BeautifulSoup(html_text, 'lxml')
    soup = BeautifulSoup(html_text, 'xml')
    
    # Parse into dict to return results, item by item
    result = {}
    
    # ARIN reports a list of CIDR net_addresses. 
    # The database will be indexed by ipaddress.net_address.
    # A record will be written for each cidr and duplicate the ARIN info
    # Obtain the organization name from tag=net instead of the tag=org which
    # has more than one tag=name making it harder to isolate.
    try:
        info = {}
        info["organization"] = fillna(soup.net.orgref["name"])
        info["handle"]       = fillna(soup.net.orgref["handle"])
        info["asn"]          = fillna(soup.net.originas)

        # Obtain rest of the info from tag=org
        info["city"]         = fillna(soup.org.city)

        # More than one address line may be recorded
        address = []
        for line in soup.org.streetaddress:
            address.append(line.string)
        info["address"]      = address

        info["postalcode"]   = fillna(soup.org.postalcode)

        # The iso3166 tags are the internation country codes
        # Ref: https://www.iso.org/glossary-for-iso-3166.html
        # The tags contain "-", illegal characters in a python var name. 
        # Use find_all to locates the tags with a string search.
        for t in soup.org.find_all("iso3166-2"):
            info["state"]    = fillna(t)
        for t in soup.org.find_all("iso3166-1"):
            info["country"]  = fillna(t.find('name'))
            countrycode = fillna(t.code2)

        # Add the risk obtained from scamalytics
        info.update(get_risk(ip_string))

        # The netblocks scope contains a list of netblock sections
        for netblock in soup.net.netblocks:
            cidr = netblock.startaddress.string + "/" + netblock.cidrlength.string      
            result[cidr] = info
    except:
        print(f"get arin error:")
        return None

    return result

class Risk():
    
    def __init__(self, filename, readonly=True):

        # Open database. Create as needed.
        
        self.readonly = readonly
        self.open_option = f'{"r" if self.readonly else "w"}'
        self.db_filename = filename
        self.hp = pickle.HIGHEST_PROTOCOL
         
        try:
            self.db = dbm.open(self.db_filename, self.open_option)
        except:
            if self.readonly:
                print(f"{self.db_filename} does not exist but will not be created when class is {readonly=}")
                return None
            else:
                self.db = dbm.open(self.db_filename, "c")
                
        # Read the data into dictionary:
        #   risk[ipaddress.ipv4network] = [organization, country, risk]
        #   
        
        self.risk = SortedDict()
        self.risk_count = 0

        for key in self.db.keys():
            self.risk[pickle.loads(key)] = pickle.loads(self.db[key])

        self.risk_count = len(self.risk)
        self.db.close()

        
    def find(self, ip_string):
        """ 
        risk[cidr] = [company, location, risk_score]
        creating one if needed and adding it to the database.
        """
        debug.prt(f"risk.find: {ip_string=}")
        import pdb; pdb.set_trace()
        try:
            self.ip = ipaddress.ip_address(ip_string)
        except:
            print(f"Could not find IPv4Address for {ip_string}")
            return None
        
        # Find the address to insert
        if self.cidr_search(self.ip):
            return self.cidr_search_result
        else:
            self.result = get_arin(ip_string)
            if self.result == None:
                debug.prt(f"No arin results for {ip_string=}\n")
                return False
            if not self.add(self.result):
                return False
            return self.result
    

    def add(self, new_risks):
        # Add the result of get_arin, a dict with cidr as key 
        # to both the Risk.risk dict and the database
        
        # Store in dictionary first.
        # There may be more than one cidr retrieved by get_arin
        # Each CIDR has to be type ip_network

        for new_cidr, new_risk in new_risks.items():
            netblock = ipaddress.ip_network(new_cidr)
            self.risk[netblock] = new_risk
        
        # Store in database next
        if not self.readonly:
            
            # Collect the pickle's of each netblock and risk
            additions = []
#             import pdb; pdb.set_trace()
            for new_cidr, new_risk in new_risks.items():
                netblock = ipaddress.ip_network(new_cidr)
                # key and value have to be pickle'd before storing
                try:
                    new_risk_temp = f"{new_risk}"
                    pickled_netblock = pickle.dumps(netblock, protocol=self.hp)
                    pickled_risk     = pickle.dumps(new_risk_temp, protocol=self.hp)
                    additions.append([pickled_netblock, pickled_risk])
                except BaseException as ex:
                    print(ex)
                    debug.prt(f"Pickle error: {new_cidr=}\n{new_risk=}\n")
                    return False

            # Write into database making sure to close it
            with dbm.open(self.db_filename, self.open_option) as self.db:
                for addition in additions:
                    self.db[addition[0]] = addition[1]

        return True
    

    def len(self):
        return self.risk_count


    def cidr_search(self, target_ip):
        # risk.cidr_search(target_ip) is True when ip's network is in db
        # type(target_ip) is ipaddress.IPv4Address
        # Updates cidr_search_result property with risk[cidr of target_ip] else None
        sz = len(self.risk)
        if sz == 0:
            self.bisearch_result = None
            return False
        s = 0
        e = sz
        while s > e:
            m = (s + e)//2
            cidr = self.risk.peekitem(m)[0]
            if target_ip in cidr:
                self.cidr_search_result = self.risk[cidr]
                return True
            if target_ip > cidr:
                s = m + 1
            else:
                e = m - 1
        self.cidr_search_result = None
        return False

# myrisk = Risk("mywhois", readonly=False)

# q = myrisk.find("142.255.122.114")

# Routine to read a clean set ip addresses from the sample data
# and icorporate them into the resk database

db_filename = "mywhois"
sample_filename = "clean_test_data.csv"

# Open the database and load the current data
risk = Risk(db_filename, readonly=False)

# Read the clean set of sample data set
clean_ip = pd.read_csv(sample_filename)
                       
# range over the unique ip addresses
new = 0
old = 0
for n, ip in enumerate(clean_ip.ip.drop_duplicates()[:10]):
    before = risk.len()
    risk.find(ip)
    after  = risk.len()
    if before == after:
        old += 1
    else:
        new += 1
print(f"{n=} {new=} {old=} {risk.len()=}")


risk.find: ip_string='142.255.122.114'

> /tmp/ipykernel_2714/2891389900.py(179)find()
    177         debug.prt(f"risk.find: {ip_string=}")
    178         import pdb; pdb.set_trace()
--> 179         try:
    180             self.ip = ipaddress.ip_address(ip_string)
    181         except:

ipdb> q


BdbQuit: 

In [ ]:
def api_arin(ip_string):
    url       = "http://whois.arin.net/rest/ip/" + ip_string + "/pft"
    html_text = ""
    try:
        html_text = requests.get(url).text
    except:
        return None
    # Parse html into a hierarchy using BeautifulSoup 
#     soup = BeautifulSoup(html_text, 'lxml')
    soup = BeautifulSoup(html_text, 'xml')
    return soup

In [ ]:
soup = api_arin("24.228.215.103")
print(f"{len(soup)=}")

In [ ]:
print(soup.netblocks)

In [ ]:
# Routine to read a clean set ip addresses from the sample data
# and icorporate them into the resk database

db_filename = "mywhois"
sample_filename = "clean_test_data.csv"

# Open the database and load the current data
risk = Risk(db_filename, readonly=False)

# Read the clean set of sample data set
clean_ip = pd.read_csv(sample_filename)
                       
# range over the unique ip addresses
new = 0
old = 0
for n, ip in enumerate(clean_ip.ip.drop_duplicates()[2:3]):
    before = risk.len()
    risk.find(ip)
    after  = risk.len()
    if before == after:
        old += 1
    else:
        new += 1
print(f"{n=} {new=} {old=} {risk.len()=}")

    



In [ ]:
risk=Risk("mywhois")

In [ ]:
for ip in clean_ip.ip.drop_duplicates()[:10]:
    print(ip)

In [ ]:
work={'organization': 'MCI Communications Services, Inc. d/b/a Verizon Business', 'handle': 'MCICS', 'asn': '', 'city': 'Ashburn', 'address': ['22001 Loudoun County Pkwy'], 'postalcode': '20147', 'state': 'VA', 'country': 'United States', 'ip': '72.68.212.63', 'score': '21', 'risk': 'medium', 'risk_comment': 'IP address 72.68.212.63 is operated by Verizon Communications whose web traffic we consider to present a potentially medium fraud risk. This IP address is owned by MCI Communications Services, Inc. d/b/a Verizon Business whose web traffic we also consider to present a potentially medium fraud risk. In both cases, non-web traffic may present a different risk or no risk at all. Scamalytics see low levels of traffic from Verizon Communications across our global network, little of which we suspect to be potentially fraudulent. We have no visibility into the web traffic directly from 72.68.212.63, and therefore apply a risk score of 21/100 based on the overall risk from Verizon Communications’s IP addresses where we do have visibility.'}
pickle.dumps(f"{work}")


In [ ]:
pp.pprint(soup)

In [ ]:
for k in risk.risk:
    print(k)

In [25]:
def parse_arin(html_text, ip_string):
    
    fillna = lambda x: "" if not x else x.string if not isinstance(x, str) else x
    
    def get_streetaddress(soup):
        # More than one address line may be recorded
        address = []
        for line in soup.org.streetaddress:
            address.append(line.string)
        return address
    def get_postalcode(soup):
        return fillna(soup.org.postalcode)
    def get_city(soup):
        return fillna(soup.org.city)
    def get_handle(soup):
        return fillna(soup.org.handle)
    def get_state(soup):
        # The iso3166 tags are the internation country codes
        # Ref: https://www.iso.org/glossary-for-iso-3166.html
        # The tags contain "-", illegal characters in a python var name. 
        # Use find_all to locate the tags with a string search.
        for t in soup.org.find_all("iso3166-2"):
            info["state"] = fillna(t)
    def get_country(soup):
        for t in soup.org.find_all("iso3166-1"):
            x = fillna(t.find('name'))
        return x
    def get_countrycode(soup):
        for t in soup.org.find_all("iso3166-1"):
            x = fillna(t.code2)
        return x
    def get_organization(soup, info):
        # There are 2 tag=name in the tag=org, one for country and one for organization. 
        # The country is part of iso3166-1 so it can be isolated. Look for organization
        # by looking at both tags and selecting the one that is not equal to country.
        for t in soup.org.find_all("name"):
            if t.string != info["country"]:
                x =illna(t)
        return t
    
    # Parse into dict to return results, item by item
    result = {}

    # Parse html into a hierarchy using BeautifulSoup 
    soup = BeautifulSoup(html_text, 'lxml')

    # ARIN reports a list of CIDR net_addresses. 
    # The database will be indexed by ipaddress.net_address.
    # A record will be written for each cidr and duplicate the ARIN info
    # Obtain the organization name from tag=net instead of the tag=org which
    # has more than one tag=name making it harder to isolate.
    try:
        info = {}

        # Obtain info from tag=org

        info["address"] = get_streetaddress(soup)
        info["postalcode"] = get_postalcode(soup)
        info["state"] = get_state(soup)
        info["country"] = get_country(soup)
        info["countrycode"] = get_country_code(soup)
#         info["organization"] = get_organization(soup, info)
#         info["city"] = get_city(soup)
#         info["handle"] = get_handle(soup)

        # Add the risk obtained from scamalytics
#         info.update(get_risk(ip_string))

        # The netblocks scope contains a list of netblock sections
        for netblock in soup.net.netblocks:
            cidr = netblock.startaddress.string + "/" + netblock.cidrlength.string      
            result[cidr] = info
        return result
    
    except:
        print(f"Error in parse_arin({ip_string=}")
        print(f"{html_text=}")
        return None

def xget_arin(ip_string):
    '''Return dict for the net_address that contains this ip_string
        {"cidr": ?,
         {"organization": ? ,
          "handle": ? ,
          "city": ? ,
          "address" : ? ,
          "postalcode": ? ,
          "countrycode": ? ,
          "state": ? ,
          "country": ? ,
         }
    '''

    # Fetch the complete record from arin restful api
    # Ref: https://www.arin.net/resources/registry/whois/rws/api/#networks-and-asns
    # ip_string ... make request by ip address as a string
    # pft ......... get full record
    
    url = "http://whois.arin.net/rest/ip/" + ip_string + "/pft"
    html_text = ""
    try:
        html_text = requests.get(url).text
    except:
        return None
#     import pdb; pdb.set_trace()
    return parse_arin(html_text, ip_string)


In [26]:
ip_string = "142.255.122.114"
print(xget_arin(ip_string))





Error in parse_arin(ip_string='142.255.122.114'
html_text='<?xml version=\'1.0\'?><?xml-stylesheet type=\'text/xsl\' href=\'https://whois.arin.net/xsl/website.xsl\' ?><ns4:pft xmlns="http://www.arin.net/whoisrws/core/v1" xmlns:ns2="http://www.arin.net/whoisrws/rdns/v1" xmlns:ns3="http://www.arin.net/whoisrws/netref/v2" xmlns:ns4="http://www.arin.net/whoisrws/pft/v1"><net copyrightNotice="Copyright 1997-2021, American Registry for Internet Numbers, Ltd." inaccuracyReportUrl="https://www.arin.net/resources/registry/whois/inaccuracy_reporting/" termsOfUse="https://www.arin.net/resources/registry/whois/tou/"><registrationDate>2012-06-19T14:21:36-04:00</registrationDate><rdapRef>https://rdap.arin.net/registry/ip/142.255.0.0</rdapRef><ref>https://whois.arin.net/rest/net/NET-142-255-0-0-1</ref><ns2:delegations copyrightNotice="Copyright 1997-2021, American Registry for Internet Numbers, Ltd." inaccuracyReportUrl="https://www.arin.net/resources/registry/whois/inaccuracy_reporting/" termsOfUse=

In [ ]:
t=requests.get("http://whois.arin.net/rest/ip/" + ip_string + "/pft").text

In [ ]:
s = BeautifulSoup(t)

In [ ]:
s.org

In [ ]:
for x in s.org.find_all("iso3166-1"):
    country = x.find('name').string
print(country)

In [ ]:
for x in s.org.find_all("name"):
    if x.string != country:
        organization = x.string


print(organization)